# Linear regression in Python

In [103]:
import pandas

data = pandas.read_csv("wage.csv")
# In R: head(data)
# In Python: data.head()
print(data.head())

   year  age            maritl      race        education        jobclass  \
0  2006   18  1. Never Married  1. White     1. < HS Grad   1. Industrial   
1  2004   24  1. Never Married  1. White  4. College Grad  2. Information   
2  2003   45        2. Married  1. White  3. Some College   1. Industrial   
3  2003   43        2. Married  3. Asian  4. College Grad  2. Information   
4  2005   50       4. Divorced  1. White       2. HS Grad  2. Information   

           health health_ins   logwage        wage  
0       1. <=Good      2. No  4.318063   75.043154  
1  2. >=Very Good      2. No  4.255273   70.476020  
2       1. <=Good     1. Yes  4.875061  130.982177  
3  2. >=Very Good     1. Yes  5.041393  154.685293  
4       1. <=Good     1. Yes  4.318063   75.043154  


We want to predict `wage` based on the other variables using linear regression. There are two main packages to do linear regression in Python:

  1. `statsmodels`: its interface is very similar to R, based on formulas.
  2. `scikit-learn`: the main package for machine learning.
  
We will look at both packages in order.

Let's start by spliting our dataset in two pieces: train and test. The dataframe `data` has 3000 rows. We will randomly select 300 of them for the test dataset (i.e. about 10%), and the rest will form the training set.

In [104]:
data_test = data.sample(n = 300, random_state = 1234)
data_train = data.drop(data_test.index)

data_train.shape[0], data_test.shape[0]

(2700, 300)

In [105]:
import statsmodels.formula.api as smf

# Note: The formula is passed as a string
fit = smf.ols('wage ~ 1', data = data_train).fit()

print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 11 Dec 2021   Prob (F-statistic):                nan
Time:                        17:37:32   Log-Likelihood:                -13954.
No. Observations:                2700   AIC:                         2.791e+04
Df Residuals:                    2699   BIC:                         2.792e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    112.2312      0.818    137.244      0.0

In [106]:
pred_vals = fit.predict(data_test)
print(pred_vals)

1936    112.23118
85      112.23118
2045    112.23118
1230    112.23118
2676    112.23118
          ...    
613     112.23118
2236    112.23118
2141    112.23118
576     112.23118
842     112.23118
Length: 300, dtype: float64


In [107]:
# RMSE
import numpy as np

actual_vals = data_test['wage']
np.sqrt(np.mean((pred_vals - actual_vals)**2))

34.14792619490065

In [108]:
# MAPE
100*np.mean(np.abs((pred_vals - actual_vals)/actual_vals))

29.44715188917207

Now let's use `age` as a covariate and see if we can improve the prediction accuracy.

In [109]:
fit2 = smf.ols('wage ~ age', data = data_train).fit()
pred_vals2 = fit2.predict(data_test)
rmse2 = np.sqrt(np.mean((pred_vals2 - actual_vals)**2))
mape2 = 100*np.mean(np.abs((pred_vals2 - actual_vals)/actual_vals))

rmse2, mape2

(32.95351934958674, 27.93519019628443)

Just like in the lecture, we can try to add more covariates and see what happens.

In [110]:
fit3 = smf.ols('wage ~ age + education', data = data_train).fit()
pred_vals3 = fit3.predict(data_test)
rmse3 = np.sqrt(np.mean((pred_vals3 - actual_vals)**2))
mape3 = 100*np.mean(np.abs((pred_vals3 - actual_vals)/actual_vals))

rmse3, mape3

(29.50978032571581, 24.23531208414976)

It's also always a good idea to write a function to avoid code repetition. Let's create a function that computes both the RMSE and the MAPE.

In [111]:
def compute_metrics(predicted, actual = actual_vals):
    rmse = np.sqrt(np.mean((predicted - actual)**2))
    mape = 100*np.mean(np.abs((predicted - actual)/actual))
    
    return rmse, mape

compute_metrics(pred_vals3)

(29.50978032571581, 24.23531208414976)

Unfortunately, `statsmodels` doesn't allow the `.` notation for selecting all variables, so we need to be explicit.

In [112]:
fit4 = smf.ols('wage ~ year+age+maritl+race+education+jobclass+health+health_ins', 
               data = data_train).fit()
pred_vals4 = fit4.predict(data_test)

compute_metrics(pred_vals4)

(27.82949112396453, 23.057299878318265)

As we can see, adding all remaining variables didn't lead to a large improvement.

**Question**: Why didn't we include the last variable, `logwage`?

### Exercise

Fit a linear model with `age`, `maritl`, `education` and `jobclass`. Compute the RMSE and MAPE. Is this model any good?

In [113]:
# Write your code here



## Splines

Let's see how we can use splines in this linear model. We will use splines with `age`, the only continuous covariate.

In [114]:
fit_spl = smf.ols('wage ~ bs(age, knots=(30, 50), degree=3)', data_train).fit()

In [115]:
pred_vals_spl = fit_spl.predict(data_test)

compute_metrics(pred_vals_spl)

(32.356480486187785, 27.003521859155928)

In [116]:
# Compare to model with only age
compute_metrics(pred_vals2)

(32.95351934958674, 27.93519019628443)

## Regularized regression

Before discussing regularized linear regression in Python, we will quickly show how to perform (classical) linear regression with `scikit-learn`.

In [117]:
from sklearn.preprocessing import OneHotEncoder
data.dtypes

year            int64
age             int64
maritl         object
race           object
education      object
jobclass       object
health         object
health_ins     object
logwage       float64
wage          float64
dtype: object

In [118]:
X = data.select_dtypes(include=[object])
# 1. instantiate the encoder
enc = OneHotEncoder(drop = 'first')
# 2. fit
enc.fit(X)
# 3. Transform
X_cat = enc.transform(X).toarray()
# Add back age
age_vec = data['age'].to_numpy().reshape((3000, 1))
X = np.hstack((age_vec, X_cat))

# Extract wage
y = data['wage'].to_numpy()

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=1234
)

X_test.shape, y_test.shape

((300, 15), (300,))

In [120]:
# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# Compute evaluation metrics
compute_metrics(y_pred, y_test)

(28.104997435587798, 23.243208445997535)

In [121]:
# Note: The intercept is automatically estimated
regr.intercept_

66.34124901950348

In [122]:
from sklearn.linear_model import Ridge
# Create linear regression object
regr_ridge = Ridge(alpha = 1.0)

# Train the model using the training sets
regr_ridge.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_ridge = regr_ridge.predict(X_test)

# Compute evaluation metrics
compute_metrics(y_pred_ridge, y_test)

(28.102403690168693, 23.24186508466164)

In [123]:
from sklearn.linear_model import Lasso
# Create lasso regression object
regr_lasso = Lasso(alpha = 1.0)

# Train the model using the training sets
regr_lasso.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_lasso = regr_lasso.predict(X_test)

# Compute evaluation metrics
compute_metrics(y_pred_lasso, y_test)

(28.483446151296956, 23.87019919968566)

Lasso regression has the special property that it can provide estimates that are exactly zero.

In [124]:
regr_lasso.coef_

array([  0.36603283,  11.79356908,  -0.        ,  -0.        ,
         0.        ,  -0.        ,   0.        ,  -0.        ,
        -5.37865231,   0.        ,  12.44049747,  32.52175758,
         2.56516129,   4.35302478, -16.47575325])

In [125]:
regr_lasso.coef_ == 0.0

array([False, False,  True,  True,  True,  True,  True,  True, False,
        True, False, False, False, False, False])